# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

# 1 $n$-gram Language Model

In [ ]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/test.txt

## 1.1 Building vocabulary

### Code

In [2]:

"""
description: Building Vocabulary
"""

from collections import defaultdict
import numpy as np

V_uni = defaultdict(int) # set default number 0 to all tokens
V_bi = defaultdict(int) # set default number 0 to all tokens
V_tri = defaultdict(int) # set default number 0 to all tokens
N_train = 0 # number of tokens
num_sentences = 0 # num of sentences in training data (number of <START> and <END> at the same time)
train_corpus_amount = 0

def preprocess(vo_dict): # filter process -> <UNK>
    # TODO: use np.where to filter, instead of "for"
    voca_list = list(vo_dict.items())  
    sum_unk = 0
    voca_key, voca_values = zip(*voca_list)
    index = np.where(np.array(voca_values)>=3)
    valid_list = list(np.array(voca_list)[index])

    invalid_index = np.where(np.array(voca_values)<3)
    invalid_list = list(np.array(voca_list)[invalid_index])
    invalid_list = [int(i[1]) for i in invalid_list]
    sum_unk = sum(invalid_list)
    valid_list.append(('<UNK>', sum_unk))
    valid_list.append(('<START>', num_sentences)) # bi-gram will use it
    valid_list.append(('<END>', num_sentences))
    return valid_list

# c(w) -- convert Vocabulary into different formats(list\ndarray\dict)
with open("./data/lm/train.txt",'r+',encoding='utf-8') as f:
    for line in f.readlines():
        curline = line.strip().split(" ") # use function "strip" to drop "\n"
        for item in curline: 
            V_uni[item] += 1
        num_sentences += 1

uni_list = preprocess(V_uni)
uni_dict = dict(uni_list)

values = uni_dict.values()
N_train = sum([int(i) for i in values ]) 
N_train -= 2*num_sentences # token number in training data, without <START> and <END>
train_corpus_amount = len(uni_list)
print("vocabulary size is", train_corpus_amount)


# c(w,u)\c(w,u,v) -- adjacent tokens pair\tri-pair
with open("./data/lm/train.txt",'r+',encoding='utf-8') as f:
    for line in f.readlines():
        curline = line.strip().split(" ") # use function "strip" to drop "\n"
        for index in range(0, len(curline)+1):
            if (index!=len(curline)) and (curline[index] not in uni_dict.keys()): # should be <UNK>
                curline[index] = '<UNK>'
            
            string_bi = None
            string_tri = None
            if index == 0:
                string_bi = curline[index]+"|"+'<START>'
                string_tri = curline[index]+"||"+'<START>'+"|"+'<START>'
            elif index == 1:
                string_bi = curline[index] + '|'+curline[index-1]
                string_tri = curline[index]+'||'+curline[index-1]+'|'+'<START>'
            elif index == len(curline):
                string_bi = '<END>'+'|'+curline[index-1]
                string_tri = '<END>'+'||'+curline[index-1]+'|'+curline[index-2]
            else:
                string_bi = curline[index] + '|' + curline[index-1]
                string_tri = curline[index] + "||" + curline[index-1] + "|" + curline[index-2] 
            V_bi[string_bi] += 1
            V_tri[string_tri] += 1

temp_bi = list(V_bi.items())
temp_bi.append(('<START>|<START>', num_sentences)) # tri-gram will use it
temp_bi.append(('<START>|<START>', num_sentences))
bi_list = temp_bi
bi_dict = dict(bi_list)

tri_list = list(V_tri.items())
tri_dict = dict(tri_list)

# Tag: uni\bi\tri_dict ------ frequency (counting number)



vocabulary size is 26601


### Discussion：
discuss the number of parameters of n-gram models.

TBD

1. 可以对所有大小写统一，统一成小写（26601-》24106）。Section 3完成
2. 需要对由于UNK（次数<3次就计入UNK）、START\END（每一句话都加一个START、END）所导致的特殊字符频率过高的问题进行讨论》》分析 or 可视化 概率，来看特殊字符的影响！
3. 在每一节的discussion的位置给出解决问题的描述 & 结果分析【用不用第三方库都是可以的】

## 1.2 $n$-gram Language Modeling

### Code

Assumption:
1. unigram perplexity $$\frac{1}{M}\sum_{i=1}^M \sqrt[N_i]{\prod_{i=1}^{N_i}{\frac{1}{p(w_i)}} }  $$
2. bigram perplexity $$\frac{1}{M}\sum_{i=1}^M \sqrt[N_i]{\prod_{i=1}^{N_i}{\frac{1}{p(w_i|w_{i-1})}} }  $$
3. trigram perplexity $$\frac{1}{M}\sum_{i=1}^M \sqrt[N_i]{\prod_{i=1}^{N_i}{\frac{1}{p(w_i|w_{i-1},w_{i-2})}} }  $$
   
tip: Perplexities in Section 1 are not taken into account

In [3]:
import copy
import math

class N_Gram():
    """
    Description: uni/bi/tri-gram model 
    Parameters: 
        - indicator: indicate the type of current object 
            [feasible values: unigram, bigram, trigram, linear_interpolation]
        - smoothing: smoothing method
            [feasible values: None, add-one, add-k, linear_interpolation]
        - k: hyperparameter when use add-k smoothing
        - uni_model: probability of each uni-words
        - bi_model:  probability of each bi-pairs
        - tri_model: probability of each tri-pairs
        - l_i_model: probability of each tri-pairs in linear_interpolation model
    """
    def __init__(self, indicator, smoothing=None, k=None) -> None:
        self.indicator = indicator # which model: unigram or bigram or trigram
        self.smoothing = smoothing # which smoothing method to be selected
        self.k = k # add-k smoothing
        self.uni_model = None
        self.bi_model = None
        self.tri_model = None
        self.l_i_model = None
        """self.train_in = 0
        self.dev_in = 0
        self.train_notin = 0
        self.dev_notin = 0"""

# Tag: self.uni\bi\tri_model ------ probability

    def train(self):
        if self.smoothing == None:
            return self.no_smoothing_training() # return a dictionary of unigram or bigram 
        elif self.smoothing == "add-one":
            return self.add_one_training()
        elif self.smoothing == "add-k":
            return self.add_k_training()
        elif self.smoothing == 'linear_interpolation':
            return self.linear_interpolation_training()
        
    def no_smoothing_training(self):
        if self.indicator == "unigram": # model: {token: probability}
            self.uni_model = copy.deepcopy(uni_dict) # frequency
            for (token, fre) in uni_dict.items():
                pro = int(fre) / N_train
                self.uni_model[token] = pro # "probability"
            return self.uni_model
            
        elif self.indicator == "bigram": # model: {token|token-1: probability}
            self.bi_model = copy.deepcopy(bi_dict)
            for (token, fre) in bi_dict.items():
                sep_index = token.find('|')
                last_token = token[sep_index+1 : ]
                if last_token not in uni_dict.keys(): 
                    pro = int(fre) / int(uni_dict['<UNK>'])
                else:
                    pro = int(fre) / int(uni_dict[last_token])
                self.bi_model[token] = pro # "probability"
            return self.bi_model

        elif self.indicator == "trigram": # model: {token||token-1|token-2: probability}
            self.tri_model = copy.deepcopy(tri_dict)
            for (token, fre) in tri_dict.items():
                sep_index = token.find('||')
                last_two_token = token[sep_index+2 : ]
                # BUG: "find" function find the first index of "||"
                if last_two_token not in bi_dict.keys():
                    pro = int(fre) / int(bi_dict['<UNK>|<UNK>'])
                else:
                    pro = int(fre) / int(bi_dict[last_two_token])
                self.tri_model[token] = pro # "probability"
            return self.tri_model
            
    def Perplexity(self,file="train"):
        name_path = {"train":"./data/lm/train.txt", "dev":"./data/lm/dev.txt", "test":"./data/lm/test.txt"}
        path = name_path[file]

        per_sum = 0
        with open(path,"r+",encoding="utf-8") as f:
            sen_num = 0
            for line in f.readlines():
                sen_num += 1
                curline = line.strip().split(" ") 
                multiply = 1
                for index in range(0,len(curline)+1):
                    if self.indicator == "unigram":
                        if index == len(curline):
                            continue
                        if curline[index] not in self.uni_model.keys():
                           curline[index] = '<UNK>' 
                        # 【此时，unigram永远不会是0！！！】
                        # 【如果验证集、测试集中，遇到了没有遇到过的token（不是frequency<3）应如何处理】
                        # A:验证集 or 测试集如果遇到了没有遇到的，同样应给成<UNK>!![而不是给成0！]
                        multiply *= self.uni_model[curline[index]]



                    elif self.indicator == "bigram":
                        # preprocess: convert to UNK
                        later_token = ''
                        if index != len(curline) and curline[index] not in uni_dict.keys():
                            curline[index] = '<UNK>'
                        # generate string
                        if index == 0:
                            string = curline[index]+'|'+'<START>'  
                            later_token = '<START>'              
                        elif index == len(curline):
                            string = '<END>'+'|'+curline[index-1]
                            later_token = curline[index-1]
                        else:
                            string = curline[index]+'|'+curline[index-1]
                            later_token = curline[index-1]
                        
                        if self.smoothing == None: # un-seen token (do not assign probability to it)
                            if string not in self.bi_model.keys(): 
                                multiply = 0
                                sen_num -= 1 
                                break
                            else: 
                                multiply *= self.bi_model[string]
                            
                        elif self.smoothing == "add-one":
                            if string not in self.bi_model.keys(): 
                                multiply = int(bi_dict['<UNK>|<UNK>']) + 1 / (int(uni_dict[later_token]) + train_corpus_amount)
                                """if file == "train":
                                    self.train_notin += 1
                                elif file == "dev":
                                    self.dev_notin += 1"""
                            else: 
                                multiply *= self.bi_model[string]
                                """if file == "train":
                                    self.train_in += 1
                                elif file == "dev":
                                    self.dev_in += 1"""

                        elif self.smoothing == "add-k":
                            if string not in self.bi_model.keys(): 
                                multiply = int(bi_dict['<UNK>|<UNK>']) + self.k / (int(uni_dict[later_token]) + self.k*train_corpus_amount)
                            else: 
                                multiply *= self.bi_model[string]
                    


                    elif self.indicator == "trigram" or self.indicator == "linear_interpolation":
                        if self.indicator == "trigram":
                            current_model = self.tri_model
                        elif self.indicator == "linear_interpolation":
                            current_model = self.l_i_model

                        if index != len(curline) and curline[index] not in uni_dict.keys():
                            curline[index] = '<UNK>'
                        if index == 0:
                            string = curline[index]+'||'+'<START>'+'|'+'<START>'   
                        elif index == 1:
                            string = curline[index]+'||'+curline[index-1]+'|'+'<START>'
                        elif index == len(curline):
                            string = '<END>'+'||'+ curline[index-1]+'|'+curline[index-2]
                        else: 
                            string = curline[index]+'||'+curline[index-1]+'|'+curline[index-2]

                        if string not in current_model.keys(): 
                            multiply = 0
                            sen_num -= 1 
                            break
                        else: 
                            multiply *= current_model[string]
              

                if multiply == 0: # exists unseen bi\tri-pairs
                    continue
                token_value = math.pow(1/multiply,1/len(curline))
                if token_value < 10000: # longlonglong sentences
                    per_sum += token_value
                # print("multiply",multiply,"len",len(curline),"value",token_value)
            #print("train_in=",self.train_in,"train_notin=",self.train_notin,
                #"dev_in=",self.dev_in,"dev_notin=",self.dev_notin,)
            return (1/sen_num)*per_sum

    def add_one_training(self):
        return "undefined add_one model yet"
    
    def add_k_training(self):
        return "undefined add_k model yet"
    
    def linear_interpolation_training(self):
        return "undefined linear_interpolation model yet"

            


# Training Phase
u_m = N_Gram(indicator='unigram', smoothing=None)
b_m = N_Gram(indicator='bigram', smoothing=None)
t_m = N_Gram(indicator='trigram', smoothing=None)
u_t = u_m.train()
b_t = b_m.train()
t_t = t_m.train()

u_ple_train = u_m.Perplexity(file="train")
b_ple_train = b_m.Perplexity(file="train")
t_ple_train = t_m.Perplexity(file="train")


# Testing Phase
print("Train: unigram model's perplexity with no smoothing method is {0}".format(u_ple_train))
print("Train: bigram model's perplexity with no smoothing method is {0}".format(b_ple_train))
print("Train: trigram model's perplexity with no smoothing method is {0}".format(t_ple_train))
print('\n')
u_ple_dev = u_m.Perplexity(file="dev")
b_ple_dev = b_m.Perplexity(file="dev")
t_ple_dev = t_m.Perplexity(file="dev")
print("Dev: unigram model's perplexity with no smoothing method is {0}".format(u_ple_dev))
print("Dev: bigram model's perplexity with no smoothing method is {0}".format(b_ple_dev))
print("Dev: trigram model's perplexity with no smoothing method is {0}".format(t_ple_dev))

# TODO；不同数据集对效果的影响！


Train: unigram model's perplexity with no smoothing method is 1261.1489758884582
Train: bigram model's perplexity with no smoothing method is 98.14200160020923
Train: trigram model's perplexity with no smoothing method is 9.829401146863853


Dev: unigram model's perplexity with no smoothing method is 1158.1941706978978
Dev: bigram model's perplexity with no smoothing method is 83.82548692125181
Dev: trigram model's perplexity with no smoothing method is 32.5206511202772


### Discussion： 
1. discuss the experimental results
2. If you encounter any problems, please analyze them and explain why.

1. 由于删除了部分token（频率<3），部分算出来概率>1
2. 困惑度不满足unigram > bigram > trigram的原因，可能是由于没有smoothing，一句话里只要有没见过的二元、三元组，bigram、trigram就给丢掉【可能这句话很有把握，但是就一个词没有见到，就被舍弃】

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing [training set and dev set]

1. compare perplexity between bigram without smoothing and bigram with add-one smoothing

2. report perplexity on training data and dev


### Code

In [4]:

class Bigram_add_one(N_Gram): # only implement add-one smoothing on bi-gram
    def add_one_training(self):
        if self.indicator != "bigram":
            return super().add_one_training()
        else:
            self.bi_model = copy.deepcopy(bi_dict)
            for (token, fre) in bi_dict.items():
                sep_index = token.find('|')
                last_token = token[sep_index+1 : ]
                if last_token not in uni_dict.keys(): # frenquency < 3
                    last_token = '<UNK>'

                pro = (int(fre) + 1) / (int(uni_dict[last_token]) + train_corpus_amount)
                self.bi_model[token] = pro # "probability"
            return self.bi_model

test_1 = Bigram_add_one(indicator='bigram', smoothing="add-one")
bi_t = test_1.train()
ple_train = test_1.Perplexity("train")
ple_dev = test_1.Perplexity("dev")
print("Train: bigram model's perplexity with add-one smoothing method is {0}".format(ple_train))
print("Dev: bigram model's perplexity with add-one smoothing method is {0}".format(ple_dev))


Train: bigram model's perplexity with add-one smoothing method is 2225.6414136546873
Dev: bigram model's perplexity with add-one smoothing method is 55.465280325390076


#### Discussion

1. Q: 基本差距不大，可能是因为多乘了一个1/fre+V的概率和sentence个数多了一个能抵消？
2. Q: bigram使用add-one和不使用add-one在训练集上性能差距过大，在验证集上性能几乎不变
   A: train.txt中的bi-gram有1616746个，由于都在训练模型时各个bigram概率都使用add-one修正，在计算perplexity的时候的所有概率全部都被缩小
   dev.txt中bi-gram一共有323816个，其中训练时得到的bi_dict覆盖了256528个，未覆盖67288个。相比train.txt中bigram个数量级差别较大，导致原来不使用smoothing method时两个困惑度基本相同，但在使用add-one后由于train.txt中pairs个数很多，就会多乘很多比原来小的概率，进而比dev.txt差很多啦！

#### 1.3.2 Add-k smoothing [training and dev set]

##### Code

In [5]:
class Bigram_add_k(N_Gram): # only implement add-one smoothing on bi-gra
    def add_k_training(self):
        if self.indicator != "bigram":
            return super().add_one_training()
        else:
            self.bi_model = copy.deepcopy(bi_dict)
            for (token, fre) in bi_dict.items():
                sep_index = token.find('|')
                last_token = token[sep_index+1 : ]
                if last_token not in uni_dict.keys(): # frenquency < 3
                    last_token = '<UNK>'

                pro = (int(fre) + self.k) / (int(uni_dict[last_token]) + self.k * train_corpus_amount)
                self.bi_model[token] = pro # "probability"
            return self.bi_model
k_list = [0.5, 0.05, 0.01]
for cur_k in k_list:
    test = Bigram_add_k(indicator='bigram', smoothing="add-k", k=cur_k)
    bi_t = test.train()
    ple_train = test.Perplexity("train")
    ple_dev = test.Perplexity("dev")
    print("Current value of k is:",cur_k)
    print("Train: bigram model's perplexity with add-k smoothing method is {0}".format(ple_train))
    print("Dev: bigram model's perplexity with add-k smoothing method is {0}".format(ple_dev))

Current value of k is: 0.5
Train: bigram model's perplexity with add-k smoothing method is 1503.7395343096173
Dev: bigram model's perplexity with add-k smoothing method is 36.74802184519655
Current value of k is: 0.05
Train: bigram model's perplexity with add-k smoothing method is 424.1169438774384
Dev: bigram model's perplexity with add-k smoothing method is 12.948378779162084
Current value of k is: 0.01
Train: bigram model's perplexity with add-k smoothing method is 215.32164031501006
Dev: bigram model's perplexity with add-k smoothing method is 8.256363243235695


##### Discussion

Briefly discuss the differences between add-k models (3 different k) and add-one models [on bigram]


### 1.3.3 Linear Interpolation [optimize on dev set, report perplexity on test set]

1. Report its perplexity on the training set and dev set for the values $\lambda_1$ = 0.2, $\lambda_2$ = 0.3, $\lambda_3$ = 0.5.
2. Please optimize on a dev set by trying different hyperparameter sets. 
3. Finally, report the perplexity on the test set with the best hyperparameter set you get. Briefly discuss the results.


#### Code

In [10]:
# --- task 1 ---
lam_1 = 0.2
lam_2 = 0.3
lam_3 = 0.5


class Linear_Interpolation(N_Gram): # only implement add-one smoothing on bi-gra
    def linear_interpolation_training(self):
        const_uni_model = None
        const_bi_model = None
        const_tri_model = None

        models = ["unigram", "bigram", "trigram"]
        for cur_type in models:
            self.indicator = cur_type
            if cur_type == "unigram": 
                temp = self.no_smoothing_training()
                const_uni_model = copy.deepcopy(temp)
            elif cur_type == "bigram": 
                temp = self.no_smoothing_training()
                const_bi_model = copy.deepcopy(temp)
            elif cur_type == "trigram": 
                temp = self.no_smoothing_training()
                const_tri_model = copy.deepcopy(temp)
        self.indicator = 'linear_interpolation'
        self.l_i_model = copy.deepcopy(self.tri_model)

        for (token, fre) in self.tri_model.items():
            index_1 = token.find('||')
            first = token[:index_1]
            last_two = token[index_1+2:]
            index_2 = last_two.find('|')
            second = last_two[:index_2]
            uni_string = first
            bi_string = first + '|' +second
            tri_string = token

            pro_uni, pro_bi, pro_tri = 0, 0, 0
            if uni_string not in self.uni_model.keys(): pro_uni = self.uni_model['<UNK>']
            else:pro_uni = self.uni_model[uni_string]
            if bi_string not in self.bi_model.keys():pro_bi = self.bi_model['<UNK>|<UNK>']
            else:pro_bi = self.bi_model[bi_string]
            if tri_string not in self.tri_model.keys(): pro_tri = self.tri_model['<UNK>||<UNK>|<UNK>']
            else:pro_tri = self.tri_model[tri_string]
        
            pro = lam_1*pro_uni + lam_2*pro_bi + lam_3*pro_tri
            self.l_i_model[token] = pro # "probability"
        return self.l_i_model,const_uni_model,const_bi_model,const_tri_model

l_i_model = Linear_Interpolation(indicator='linear_interpolation', smoothing='linear_interpolation')
l_i,const_uni_model,const_bi_model,const_tri_model\
     = l_i_model.train()
ple_train = l_i_model.Perplexity("train")
ple_dev = l_i_model.Perplexity("dev")
print('----- task 1: report perplexity on train & dev set with initial values of \lambda_1_2_3 -----')
print("Train: linear_interpolation model's perplexity with linear_interpolation smoothing method is {0}".format(ple_train))
print("Dev: linear_interpolation model's perplexity with linear_interpolation smoothing method is {0}".format(ple_dev))


# --- task 2 ---

feasible_value = []
for i in range(0,10):
    lam_1 = i/10
    for j in range(0,10-i+1):
        lam_2 = j/10
        lam_3 = (10-i-j)/10
        feasible_value.append((lam_1,lam_2,lam_3)) # immutable element could be dictionary's key!!!


class Optimize_linear_Interpolation(N_Gram): 
    def linear_interpolation_training(self):
        feaisble_perplexity = {}.fromkeys(feasible_value,0)

        self.l_i_model = copy.deepcopy(const_tri_model)
        for (lam_1,lam_2,lam_3) in feasible_value:
            for (token, fre) in const_tri_model.items():
                index_1 = token.find('||')
                first = token[:index_1]
                last_two = token[index_1+2:]
                index_2 = last_two.find('|')
                second = last_two[:index_2]
                uni_string = first
                bi_string = first + '|' +second
                tri_string = token

                pro_uni, pro_bi, pro_tri = 0, 0, 0
                if uni_string not in const_uni_model.keys(): pro_uni = const_uni_model['<UNK>']
                else:pro_uni = const_uni_model[uni_string]
                if bi_string not in const_bi_model.keys(): pro_bi = const_bi_model['<UNK>|<UNK>']
                else:pro_bi = const_bi_model[bi_string]
                if tri_string not in const_tri_model.keys(): pro_tri = const_tri_model['<UNK>||<UNK>|<UNK>']
                else:pro_tri = const_tri_model[tri_string]
                pro = lam_1*pro_uni + lam_2*pro_bi + lam_3*pro_tri
                self.l_i_model[token] = pro # "probability"

            cur_per = self.Perplexity(file='dev')
            feaisble_perplexity[(lam_1,lam_2,lam_3)] = cur_per
        return feaisble_perplexity # return a dict

o_l_i_model = Optimize_linear_Interpolation(indicator='linear_interpolation', smoothing='linear_interpolation')
result_dict = o_l_i_model.train()
print('\n')
result_list = list(result_dict.items())
def list_sort(item):
    return item[1]
result_list.sort(key=list_sort)
print("----- task 2: find the best hyperparameter set, along with regarding perplexity -----")
print("the best hyperparameter of (lambda_1, lambda_2, lambda_3) is: {0} respectively".format(result_list[0][0]))
print("the regarding perplexity with best hyperparameter set in dev.txt is: {0}".format(result_list[0][1]))


# --- task 3 ---
print('\n')
lam_1,lam_2,lam_3 = result_list[0][0]
test_model = Linear_Interpolation(indicator='linear_interpolation', smoothing='linear_interpolation')
l_i,const_uni_model,const_bi_model,const_tri_model\
     = test_model.train()
ple_test = test_model.Perplexity("test")
print('----- task 3: report perplexity on test set with the best values of \lambda_1_2_3 -----')
print("Test: linear_interpolation model's perplexity with linear_interpolation smoothing method is {0}".format(ple_test))





----- task 1: report perplexity on train & dev set with initial values of \lambda_1_2_3 -----
Train: linear_interpolation model's perplexity with linear_interpolation smoothing method is 16.06279614001182
Dev: linear_interpolation model's perplexity with linear_interpolation smoothing method is 31.928149799280426


----- task 2: find the best hyperparameter set, along with regarding perplexity -----
the best hyperparameter of (lambda_1, lambda_2, lambda_3) is: (0.0, 0.3, 0.7) respectively
the regarding perplexity with best hyperparameter set in dev.txt is: 26.600135677586284


----- task 3: report perplexity on test set with the best values of \lambda_1_2_3 -----
Test: linear_interpolation model's perplexity with linear_interpolation smoothing method is 27.23544219591197


#### Discussion

##### 1.3.4 Optimization

#### Discussion

# 2 Word Vectors [use pre-trained Glove]

In [36]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

Loaded vocab size 400000


## 2.1 Find most similar word
Use cosine similarity to find the most similar word to each of these words. Report the most similar word and its cosine similarity.
- dog
- whale
- before
- however
- fabricate

In [40]:
# we have wv_from_bin 
# learning material: https://blog.csdn.net/u010700066/article/details/83070102
"""
print(wv_from_bin.get_vector("dog"))  # 查看向量
wv_from_bin.key_to_index  # 查看词和对应向量
wv_from_bin.index2word  # 每个index对应的词
"""


print(type(wv_from_bin))

# --- task 1 --- 
vocabulary = ['dog', 'whale', 'before', 'however', 'fabricate']
for item in vocabulary:
    print('word:{0:<10} - most similar word:{1:<10} - regarding cosine similarity is:{2:<10}\
        '.format(item,wv_from_bin.most_similar(item)[1][0],wv_from_bin.most_similar(item)[1][1]))



<class 'gensim.models.keyedvectors.KeyedVectors'>
word:dog        - most similar word:cat        - regarding cosine similarity is:0.7444875836372375        
word:whale      - most similar word:humpback   - regarding cosine similarity is:0.702865719795227        
word:before     - most similar word:when       - regarding cosine similarity is:0.8611732125282288        
word:however    - most similar word:though     - regarding cosine similarity is:0.8910207152366638        
word:fabricate  - most similar word:invent     - regarding cosine similarity is:0.5735598802566528        


## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- france : french :: england : ?:
- france : wine :: england : ?

In [45]:

vocabulary_list = [('dog',"puppy", "cat"), ("speak", "speaker", "sing"), ('france', "french", "england"),\
    ("france", "wine", "england")]

for item in vocabulary_list:
    start_exi = item[0]; A = wv_from_bin.get_vector(start_exi)
    end_exi = item[1]; B = wv_from_bin.get_vector(end_exi)
    start_new = item[2]; C = wv_from_bin.get_vector(start_new)
    D = C + (B - A)
    print('the potential answers of analogy "{0}:{1}::{2}:?" are {3}'.format(item[0],item[1],item[2],\
        wv_from_bin.most_similar(D)[:3] ))
    print('\n')



the potential answers of analogy "dog:puppy::cat:?" are [('puppy', 0.7710572481155396), ('cat', 0.6285784840583801), ('puppies', 0.6080518960952759)]


the potential answers of analogy "speak:speaker::sing:?" are [('speaker', 0.6879462003707886), ('sing', 0.5925208926200867), ('sang', 0.5009135603904724)]


the potential answers of analogy "france:french::england:?" are [('english', 0.7608821988105774), ('england', 0.759766697883606), ('british', 0.5845818519592285)]


the potential answers of analogy "france:wine::england:?" are [('wine', 0.6376140713691711), ('tea', 0.5302954912185669), ('england', 0.5272895693778992)]




# 3 Sentiment analysis

In [ ]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

## 3.1 Logistic Regression
1. build three types of features to train a classifier: Unigram, Bigram, and GloVe
(Global Vectors forWord Representation) features. 
2. Use logistic regression to predict the sentiment labels of movie reviews. 
    You can only use train.txt to train the model. 
3. Report the P, R, F1 results of dev.txt in a table and compare the performance difference of these three features.


### Data prepocess -- generate unigram and bigram features dictionaries

In [3]:

"""
description: Building Unigram Features
"""

from collections import defaultdict
import numpy as np

V_uni = defaultdict(int) # set default number 0 to all tokens
V_bi = defaultdict(int) # set default number 0 to all tokens
sentence_label = [] # labels in train.txt
sentence_element = [] # elements of each line in train.txt
N_train = 0 # 【不去重】number of tokens [inclusive tokens which appear several times]
num_sentences = 0 # num of sentences in training data (number of <START> and <END> at the same time)
train_corpus_amount = 0 # 【去重】number of tokens [exclusive tokens appear several times]

def preprocess(vo_dict): # filter process -> <UNK>
    # TODO: use np.where to filter, instead of "for"
    voca_list = list(vo_dict.items())  
    sum_unk = 0
    voca_key, voca_values = zip(*voca_list)
    index = np.where(np.array(voca_values)>=3)
    valid_list = list(np.array(voca_list)[index])

    invalid_index = np.where(np.array(voca_values)<3)
    invalid_list = list(np.array(voca_list)[invalid_index])
    invalid_list_num = [int(i[1]) for i in invalid_list]
    sum_unk = sum(invalid_list_num)
    valid_list.append(('<UNK>', sum_unk))
    valid_list.append(('<START>', num_sentences)) # bi-gram will use it
    valid_list.append(('<END>', num_sentences))
    return valid_list,invalid_list

# Unigram Features --- c(w): convert Vocabulary into different formats(list\dict)
with open("./data/classification/train.txt",'r+',encoding='utf-8') as f:
    for line in f.readlines():
        curline = line.strip().split("\t")
        sentence_label.append(int(curline[0]))
        string = str(curline[1])
        curline = string.split(" ")
         # use function "strip" to drop "\n"
        sentence_element.append(curline)
        for index in range(0,len(curline)): 
            V_uni[curline[index]] += 1
        num_sentences += 1

uni_fre_list, unk_fre_list = preprocess(V_uni)
uni_fre_dict = dict(uni_fre_list)
unk_fre_dict = dict(unk_fre_list)

values = uni_fre_dict.values()
N_train = sum([int(i) for i in values])
N_train -= 2*num_sentences # token number in training data, without <START> and <END>
train_corpus_amount = len(uni_fre_list)
print("vocabulary size is", train_corpus_amount,"with <UNK>, <START>, <END>")


# Bigram Features: c(w,u) -- adjacent tokens pair (list\dict)
for curline in sentence_element:
    for index in range(0, len(curline)+1):
        if (index!=len(curline)) and (curline[index] not in uni_fre_dict.keys()): # should be <UNK>
            curline[index] = '<UNK>'    
        string_bi = None
        if index == 0:
            string_bi = curline[index]+"|"+'<START>'
        elif index == 1:
            string_bi = curline[index] + '|'+curline[index-1]
        elif index == len(curline):
            string_bi = '<END>'+'|'+curline[index-1]
        else:
            string_bi = curline[index] + '|' + curline[index-1]
        V_bi[string_bi] += 1

temp_bi = list(V_bi.items())
temp_bi.append(('<START>|<START>', num_sentences)) # tri-gram will use it
temp_bi.append(('<START>|<START>', num_sentences))
bi_fre_list = temp_bi
bi_fre_dict = dict(bi_fre_list)
# Tag: uni\bi\tri_dict ------ frequency (counting number)


# 可能去掉频率低的词汇会降低准确率？比如一些不常用的表达情感词语
# 可能减少一些N_train可以提升准确率？
# 可能用原来的频次可能会提升准确率？不除以所有词出现的次数？
# 特征的归一化、标准化大概率可以提升性能

vocabulary size is 5067 with <UNK>, <START>, <END>


### Unigram Features -- generate LR classifier

In [18]:
from collections import defaultdict

uni_feature = []
keys =  uni_fre_dict.keys()
unk_list = unk_fre_dict.keys()
# unk_list --- token need to be unknown 
for line in sentence_element:
    cur_dict = defaultdict(int) # frequency of each word occurs in this line
    for item in line:
        if item in unk_list:
            cur_dict['<UNK>'] += 1
        else:
            cur_dict[item] += 1
    length = len(line) # length of current line, inclusive <UNK>
    current_embedding = [] 
    for item in keys:
        if item in line: # vocabulary
            current_embedding.append(cur_dict[item]/length)
        else:
            current_embedding.append(0)
    uni_feature.append(current_embedding)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 处理1：标准化
scaler = StandardScaler() # standarize data
scaler = scaler.fit(uni_feature) # fit，本质是生成均值和方差
uni_feature = scaler.transform(uni_feature) #通过接口导出结果

# 处理2：降维
# n_components=500, score = 0.7852601156069364 0.7826589595375723
pca = PCA(n_components=500)
pca = pca.fit(uni_feature)
print(pca.explained_variance_ratio_)
new_uni_features = pca.transform(uni_feature)


In [51]:
# 处理3：预测时加入正则化项（如L1，进行特征选择）。也要尝试L2好不好
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LRCV

lrl1 = LR(penalty='l1',solver='saga', C=0.5, max_iter=2000)
lrl1 = lrl1.fit(new_uni_features, sentence_label)
lrcvl1 = LRCV(penalty='l1',solver='saga', max_iter=2000)
lrcvl1 = lrcvl1.fit(new_uni_features, sentence_label)

In [52]:
lr_score = lrl1.score(new_uni_features, sentence_label)
lrcv_score = lrcvl1.score(new_uni_features, sentence_label)
print(lr_score, lrcv_score)

0.7852601156069364 0.7826589595375723


### Bigram Features

### GloVe Features

Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |           |        |          |
| bigram      |           |        |          |
| GloVe       |           |        |          |

## 3.2 Better Feature